# Social Media and Human-Computer Interaction - Part 4

### Goal: Use social media posts to explore the appplication of text and natural language processing to see what might be learned from online interactions.

Specifically, we will retrieve, annotate, process, and interpret Twitter data on health-related issues such as smoking.

--- 
References:
* [Mining Twitter Data with Python (Part 1: Collecting data)](https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/)
* The [Tweepy Python API for Twitter](http://www.tweepy.org/)

Required Software
* [Python 3](https://www.python.org)
* [NumPy](http://www.numpy.org) - for preparing data for plotting
* [Matplotlib](https://matplotlib.org) - plots and garphs
* [jsonpickle](https://jsonpickle.github.io) for storing tweets. 
---

In [1]:
%matplotlib inline

import operator
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import jsonpickle
import json
import random
import tweepy
import spacy
import time
from datetime import datetime
from spacy.symbols import ORTH, LEMMA, POS

# 4.0 Introduction

Picking up where  [Part 3](SocialMedia - Part 3.ipynb) left off, this module introduces basics of classification of textual content, using classifiers from the popular [Scikit-learn](http://scikit-learn.org/stable/index.html) machine-learning tools. 

# 4.0.1 Setup

As before, we start with the Tweets class and the configuration for our Twitter API connection.  We may not need this, but we'll load it in any case.

In [2]:
class Tweets:
    
    
    def __init__(self,term="",corpus_size=100):
        self.tweets={}
        if term !="":
            self.searchTwitter(term,corpus_size)
                
    def searchTwitter(self,term,corpus_size):
        searchTime=datetime.now()
        while (self.countTweets() < corpus_size):
            new_tweets = api.search(term,lang="en",count=10)
            for nt_json in new_tweets:
                nt = nt_json._json
                if self.getTweet(nt['id_str']) is None and self.countTweets() < corpus_size:
                    self.addTweet(nt,searchTime,term)
            time.sleep(5)
                
    def addTweet(self,tweet,searchTime,term="",count=0):
        id = tweet['id_str']
        if id not in self.tweets.keys():
            self.tweets[id]={}
            self.tweets[id]['tweet']=tweet
            self.tweets[id]['count']=0
            self.tweets[id]['searchTime']=searchTime
            self.tweets[id]['searchTerm']=term
        self.tweets[id]['count'] = self.tweets[id]['count'] +1
        
    def getTweet(self,id):
        if id in self.tweets:
            return self.tweets[id]['tweet']
        else:
            return None
    
    def getTweetCount(self,id):
        return self.tweets[id]['count']
    
    def countTweets(self):
        return len(self.tweets)
    
    # return a sorted list of tupes of the form (id,count), with the occurrence counts sorted in decreasing order
    def mostFrequent(self):
        ps = []
        for t,entry in self.tweets.items():
            count = entry['count']
            ps.append((t,count))  
        ps.sort(key=lambda x: x[1],reverse=True)
        return ps
    
    # reeturns tweet IDs as a set
    def getIds(self):
        return set(self.tweets.keys())
    
    # save the tweets to a file
    def saveTweets(self,filename):
        json_data =jsonpickle.encode(self.tweets)
        with open(filename,'w') as f:
            json.dump(json_data,f)
    
    # read the tweets from a file 
    def readTweets(self,filename):
        with open(filename,'r') as f:
            json_data = json.load(f)
            incontents = jsonpickle.decode(json_data)   
            self.tweets=incontents
        
    def getSearchTerm(self,id):
        return self.tweets[id]['searchTerm']
    
    def getSearchTime(self,id):
        return self.tweets[id]['searchTime']
    
    def getText(self,id):
        tweet = self.getTweet(id)
        text=tweet['full_text']
        if 'retweeted_status'in tweet:
            original = tweet['retweeted_status']
            text=original['full_text']
        return text
                
    def addCode(self,id,code):
        tweet=self.getTweet(id)
        if 'codes' not in tweet:
            tweet['codes']=set()
        tweet['codes'].add(code)
        
   
    def addCodes(self,id,codes):
        for code in codes:
            self.addCode(id,code)
        
 
    def getCodes(self,id):
        tweet=self.getTweet(id)
        return tweet['codes']
    
    # NEW -ROUTINE TO GET PROFILE
    def getCodeProfile(self):
        summary={}
        for id in self.tweets.keys():
            tweet=self.getTweet(id)
            if 'codes' in tweet:
                for code in tweet['codes']:
                    if code not in summary:
                            summary[code] =0
                    summary[code]=summary[code]+1
        sortedsummary = sorted(summary.items(),key=operator.itemgetter(0),reverse=True)
        return sortedsummary

*REDACT FOLLOWING DETAILS*

In [3]:
consumer_key='D2L4YZ2YrO1PMix7uKUK63b8H'
consumer_secret='losRw9T8zb6VT3TEJ9JHmmhAmn1GXKVj30dkiMv9vjhXuiWek9'
access_token='15283934-iggs1hiZAPI2o5sfHWMfjumTF7SvytHPjpPRGf3I6'
access_secret='bOvqssxS97PGPwXHQZxk83KtAcDyLhRLgdQaokCdVvwFi'

In [4]:
from tweepy import OAuthHandler

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

We will also load some routines that we defined in [Part 3](SocialMedia - Part 3.ipynb):
    
1. Our routine for creating a customized NLP pipeline
2. Our routine for including tokens
3. The `filterTweetTokens` routine defined in an exercise

In [5]:
def getTwitterNLP():
    nlp = spacy.load('en')
    
    for word in nlp.Defaults.stop_words:
        lex = nlp.vocab[word]
        lex.is_stop = True
    
    special_case = [{ORTH: u'e-cigarette', LEMMA: u'e-cigarette', POS: u'NOUN'}]
    nlp.tokenizer.add_special_case(u'e-cigarette', special_case)
    nlp.tokenizer.add_special_case(u'E-cigarette', special_case)
    vape_case = [{ORTH: u'vape',LEMMA:u'vape',POS: u'NOUN'}]
    
    vape_spellings =[u'vap',u'vape',u'vaping',u'vapor',u'Vap',u'Vape',u'Vapor',u'Vapour']
    for v in vape_spellings:
        nlp.tokenizer.add_special_case(v, vape_case)
    def hashtag_pipe(doc):
        merged_hashtag = True
        while merged_hashtag == True:
            merged_hashtag = False
            for token_index,token in enumerate(doc):
                if token.text == '#':
                    try:
                        nbor = token.nbor()
                        start_index = token.idx
                        end_index = start_index + len(token.nbor().text) + 1
                        if doc.merge(start_index, end_index) is not None:
                            merged_hashtag = True
                            break
                    except:
                        pass
        return doc
    nlp.add_pipe(hashtag_pipe,first=True)
    return nlp

def includeToken(tok):
    val =False
    if tok.is_stop == False:
        if tok.is_alpha == True: 
            if tok.text =='RT':
                val = False
            elif tok.pos_=='NOUN' or tok.pos_=='PROPN' or tok.pos_=='VERB':
                val = True
        elif tok.text[0]=='#' or tok.text[0]=='@':
            val = True
    if val== True:
        stripped =tok.lemma_.lower().strip()
        if len(stripped) ==0:
            val = False
        else:
            val = stripped
    return val

def filterTweetTokens(tokens):
    filtered=[]
    for t in tokens:
        inc = includeToken(t)
        if inc != False:
            filtered.append(inc)
    for ent in tokens.ents:
        stripped =ent.text.strip()
        if len(stripped) >0:
            filtered.append(ent.text.strip())
    return filtered

We will start things off by reading in our two stored sets of tweets, and creating an NLP object of processing the tweets:

In [6]:
vaping=Tweets()
vaping.readTweets("tweets-vaping.json")
print("Number of vaping tweets: "+str(vaping.countTweets()))
smoking=Tweets()
smoking.readTweets("tweets-smoking.json")
print("Number of smoking tweets: "+str(smoking.countTweets()))

Number of vaping tweets: 100
Number of smoking tweets: 100


Finally, we will include some additional modules from Scikit-Learn:

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
import string
import re

# 4.0.2 An outline for classification

Our examination of basic machine learning from text will address several key tasks:
    
1. Converting text into numerical representations appropriate for machine learning algorithms
2. Creating a classification pipeline.
3. Dividing a dataset into test and train sets. 
4. Training, testing, and evaluating a model.

Much of the ideas below are informed by Nic Schrading's [Intro to NLP with spaCy](https://nicschrading.com/project/Intro-to-NLP-with-spaCy/) and the [scikit-learn Working with Text data tutorial[(http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#extracting-features-from-text-files).

# 4.1 Vectorization: converting text into numerical representation

Even when tokenzied, tweets are little more than glorified collections of text. As machine learning algorithms used for building classifiers operate on numbers, not text, we must convert the collection of words/tokens in each tweet into an alternative form capable of encoding all of the information in a tweet in numeric form. These numeric representations can be used to calculate similarities between items, thus forming the basis for comparisons used in clustering and machine learning.


## 4.1.1 Basic vectorization
The easiest way to do this is to convert each tweet into a *vector* of numbers, one for each word that might possibly show up in any of the tweets. The entry for each word will contain the number of times that that word occurs in the given tweet. This simple representation captures enough information to distinguish between tweets, at the expense of losing some information that might prove valuable for some tasks (try to think about what information a vector might not include - we'll get back to that later).  

Fortunately, the scikit learn library makes it easy to convert tokens from a tweet into a vector.  Too see how this works, let's start with the text from a few random tweets.

In [35]:
nlp=getTwitterNLP()
tweet_ids=list(smoking.getIds())[:5]
texts=[smoking.getText(t) for t in tweet_ids]

In [36]:
texts

['Welcome to Eternity...Smoking or non smoking ? https://t.co/W9nAU7GEaQ',
 'I know I been smoking myself to sleep lately cause ive been going to sleep with the lights on 😒 lol',
 'me: smoking weed hasn’t affected me at all\n\nsomeone: count to 10\n\nme: https://t.co/SUoGzARpom',
 'Made a sandwich 10 min ago and been looking for it ever since then\U0001f926🏾\u200d♂️ I gotta stop smoking😂 https://t.co/NCbNOyvZXe',
 'He was walking smoking him a blunt by me saying if u really wannabe on tv n free wit ur babies u will chose me n make him our baby maker.']

We can then take this text and run it through Scikit-learn's `CountVectorizer`, which will turn this text into a vector representation suitable for machine learning. We mut first fit and then transform the vectorizer

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
vectorizer.fit(texts)
vec =vectorizer.transform(texts)

The vectorizer has five rows (one for each entry) and one column for each unique term found in any of the entries.  W can see the terms by looking at the vocabulary:

In [38]:
print(vectorizer.vocabulary_)

{'count': 14, 'then': 54, 'looking': 32, 'wit': 65, 'ago': 2, 'weed': 62, 'ever': 16, 'all': 3, 'sleep': 48, 'lights': 30, 'suogzarpom': 52, 'our': 43, 'someone': 50, 'by': 10, 'lately': 29, 'make': 34, 'gotta': 20, 'walking': 59, 'hasn': 21, 'wannabe': 60, 'eternity': 15, 'and': 4, 'welcome': 63, 'him': 23, 'free': 18, 'blunt': 9, 'babies': 6, 'it': 26, 'non': 40, 'lol': 31, 'co': 13, 'ive': 27, 'at': 5, 'baby': 7, 'with': 66, 'know': 28, 'w9nau7geaq': 58, 'min': 37, 'or': 42, 'smoking': 49, 'maker': 35, 'to': 55, 'going': 19, 'since': 47, 'myself': 38, 'for': 17, 'saying': 46, 'me': 36, 'stop': 51, 'made': 33, 'the': 53, 'chose': 12, 'really': 44, 'been': 8, 'on': 41, 'if': 25, 'he': 22, '10': 0, 'was': 61, 'https': 24, 'ncbnoyvzxe': 39, 'tv': 56, 'will': 64, 'sandwich': 45, 'affected': 1, 'ur': 57, 'cause': 11}


And we can look at the results by printing the array version of the transformed vectorizer:

In [39]:
print(vec.toarray())

[[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 0 1 0 0 0 0 0 0 2 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 2 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0
  0 0 1 0 0 1 0 0 0 0 0 0 2 1 0 0 0 1 0 2 0 0 0 0 0 0 0 0 0 0 1]
 [1 1 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  3 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0]
 [1 0 1 0 1 0 0 0 1 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0
  0 1 0 1 0 0 0 0 0 1 0 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 2 0 1 0 0 0 0 0 0 0 0 1 1
  2 0 0 0 0 1 0 1 1 0 1 0 0 1 0 0 0 0 0 0 1 1 0 1 1 1 0 0 1 1 0]]


To interpret this , can look at the second tweet

In [40]:
texts[2]

'me: smoking weed hasn’t affected me at all\n\nsomeone: count to 10\n\nme: https://t.co/SUoGzARpom'

Notice that the word 'me' has occurred three times in this tweet. We can stat by finding the position of 'me' in the vocabulary:

In [41]:
index = vectorizer.vocabulary_.get("me")
print(index)

36


so column 36 has the count for 'me'. Let's look at the value in row 2, column 36.

In [42]:
vec[2,36]

3

the count is 3, as expected! 

Try a few more examples to confirm that the counts are working. 

## 4.1.2 TF-IDF Vectorization

Vectorization by count is only one approach that we might take. However, direct counts suffer from an important shortcoming: if we leave in all of the prepositions and all other low-content words, we might see vectors that are dominated by these words, making comparisons based on more informative words more difficult.  More generally, if we have texts with very similar wods, the counts might obscure some of the key differences. 


To see how this might work, try the following experiment:

Imagine these two texts:

* "The man went to the store to by some milk."
* "The man went to the store to by some coffee."

Clearlly, the most interesting difference here is 'milk' vs. 'coffee'. Let's see what happens when we try to vectorize these texts. 

In [48]:
simtexts=["The man went to the store to by some milk.","The man went to the store to by some coffee."]
simvectorizer=CountVectorizer()
simvectorizer.fit(texts)
simvec =vectorizer.transform(simtexts)
print(simvec.toarray())

[[1 0 1 1 1 1 2 2 1]
 [1 1 1 0 1 1 2 2 1]]


Note that these vectors are almost idenical, differing only in the second and fourth positions.  We can guess that one of these corresponds to 'milk' and the other to 'coffee'. Let's confirm:

In [50]:
print(simvectorizer.vocabulary_.get("milk"))
print(simvectorizer.vocabulary_.get("coffee"))

3
1


Confirmed. For some applications, this similarity might be fine. For others, it might be undesirable. 

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer